## predict on an image or a video

In [11]:
# !yolo task=detect mode=predict model={model_path} conf={confidence_threshold} source={source_path} save={save_output}

In [5]:

!yolo task=detect mode=predict model=../models/best.pt conf=0.25 source=../dataset/test/images/621_jpg.rf.6f6b36ba93a06a78068271c079fb91d0.jpg save=True

Ultralytics YOLOv8.0.196  Python-3.10.0 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

image 1/1 C:\Users\Jarvis\Desktop\Web\Fire_Detection\Code\..\dataset\test\images\621_jpg.rf.6f6b36ba93a06a78068271c079fb91d0.jpg: 800x800 1 fire - v1 2024-01-12 8-02pm, 18.9ms
Speed: 8.1ms preprocess, 18.9ms inference, 5.7ms postprocess per image at shape (1, 3, 800, 800)
Results saved to runs\detect\predict2
 Learn more at https://docs.ultralytics.com/modes/predict


## just give the path to the video at source

In [1]:

!yolo task=detect mode=predict model=../models/best.pt conf=0.25 source=../dataset/other_data/video1.mp4 save=True

Ultralytics YOLOv8.0.196  Python-3.10.0 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

video 1/1 (1/522) C:\Users\Jarvis\Desktop\Web\Fire_Detection\Code\..\dataset\other_data\video1.mp4: 480x800 (no detections), 124.1ms
video 1/1 (2/522) C:\Users\Jarvis\Desktop\Web\Fire_Detection\Code\..\dataset\other_data\video1.mp4: 480x800 (no detections), 56.0ms
video 1/1 (3/522) C:\Users\Jarvis\Desktop\Web\Fire_Detection\Code\..\dataset\other_data\video1.mp4: 480x800 (no detections), 25.0ms
video 1/1 (4/522) C:\Users\Jarvis\Desktop\Web\Fire_Detection\Code\..\dataset\other_data\video1.mp4: 480x800 (no detections), 18.6ms
video 1/1 (5/522) C:\Users\Jarvis\Desktop\Web\Fire_Detection\Code\..\dataset\other_data\video1.mp4: 480x800 (no detections), 18.1ms
video 1/1 (6/522) C:\Users\Jarvis\Desktop\Web\Fire_Detection\Code\..\dataset\other_data\video1.mp4: 480x800 (no detections), 18.1ms
video 1/1 (7/5

## Visualize the Prediction on a single image

In [22]:
import os
import cv2
from ultralytics import YOLO

# Initialize the YOLO model
model_path = "../models/best.pt"
model = YOLO(model_path)

# Load an image
image_path = "../dataset/test/images/new_fire_fire-538_png_jpg.rf.a0a8afd83ed3d773494b3da41d6e390b.jpg"
img = cv2.imread(image_path)

# Perform prediction
results = model.predict([img])

# Assuming 'results[0].boxes' contains the detection results for the first (and in this case, only) image
detections = results[0].boxes

# Check if any detections were made
if detections.data.shape[0] > 0:
    for i in range(detections.data.shape[0]):
        box = detections.data[i]
        x1, y1, x2, y2 = map(int, box[:4])  # Extract bounding box coordinates
        conf = box[4].item()  # Confidence score

        # Draw the bounding box
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 4)

        # Prepare the label as "Fire" and the confidence score
        label = f'Fire: {conf:.2f}'

        # Calculate text width & height to draw the transparent box as background of the text
        (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)

        # Display the label at the top of the bounding box
        label_position = (x1, y1 - 15 if y1 - 15 > 15 else y1 + 15)
        cv2.rectangle(img, (x1, label_position[1] - h - 3), (x1 + w, label_position[1] + h - 3), (0, 255, 0), -1)
        cv2.putText(img, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 0), 2)

# Display the image with detections
cv2.imshow("Detections", img)
cv2.waitKey(0)  # Wait for a key press to exit
cv2.destroyAllWindows()

# Save the result
# Extract filename and extension
filename = os.path.basename(image_path)
output_path = f"../results/{filename}"
cv2.imwrite(output_path, img)
print(f"Image saved to {output_path}")



0: 800x800 2 fire - v1 2024-01-12 8-02pms, 19.0ms
Speed: 5.0ms preprocess, 19.0ms inference, 3.0ms postprocess per image at shape (1, 3, 800, 800)


Image saved to ../results/new_fire_fire-538_png_jpg.rf.a0a8afd83ed3d773494b3da41d6e390b.jpg


## give path to the video, it will be processed and stored in results

In [2]:
import os
import cv2
from ultralytics import YOLO

# Initialize the YOLO model
model_path = "../models/best.pt"
model = YOLO(model_path)

# Load a video
video_path = "../dataset/other_data/video1.mp4"
cap = cv2.VideoCapture(video_path)

# Extract filename without extension and add new path
filename_wo_extension = os.path.splitext(os.path.basename(video_path))[0]
output_filename = f"{filename_wo_extension}_detections.mp4"
output_path = os.path.join('../results', output_filename)

# Define the codec and create VideoWriter object to save the video
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter(output_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform prediction
    results = model.predict([frame])

    # Assuming 'results[0].boxes' contains the detection results for the first (and in this case, only) image
    detections = results[0].boxes

    # Check if any detections were made
    if detections.data.shape[0] > 0:
        for i in range(detections.data.shape[0]):
            box = detections.data[i]
            x1, y1, x2, y2 = map(int, box[:4])  # Extract bounding box coordinates
            conf = box[4].item()  # Confidence score

            # Draw the bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 4)

            # Prepare the label as "Fire" and the confidence score
            label = f'Fire: {conf:.2f}'

            # Display the label at the top of the bounding box
            label_position = (x1, y1 - 15 if y1 - 15 > 15 else y1 + 15)
            (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)
            cv2.rectangle(frame, (x1, label_position[1] - h - 3), (x1 + w, label_position[1]), (0, 255, 0), -1)
            cv2.putText(frame, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 0), 2)

    # Write the frame into the file
    out.write(frame)

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()
print(f"The video was successfully processed and saved as {output_path}.")



0: 480x800 (no detections), 106.9ms
Speed: 5.2ms preprocess, 106.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 (no detections), 12.5ms
Speed: 4.5ms preprocess, 12.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 (no detections), 14.2ms
Speed: 4.3ms preprocess, 14.2ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 (no detections), 15.5ms
Speed: 4.5ms preprocess, 15.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 (no detections), 14.9ms
Speed: 5.8ms preprocess, 14.9ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 (no detections), 14.2ms
Speed: 4.0ms preprocess, 14.2ms inference, 3.1ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 (no detections), 16.0ms
Speed: 3.5ms preprocess, 16.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 (no detections), 14.6ms
Speed: 4.2ms preprocess, 14.6ms


0: 480x800 (no detections), 13.0ms
Speed: 4.0ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 (no detections), 11.5ms
Speed: 4.0ms preprocess, 11.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 (no detections), 11.0ms
Speed: 4.5ms preprocess, 11.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 (no detections), 11.5ms
Speed: 5.0ms preprocess, 11.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 (no detections), 12.0ms
Speed: 3.5ms preprocess, 12.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 (no detections), 12.0ms
Speed: 3.2ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 (no detections), 11.8ms
Speed: 4.0ms preprocess, 11.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 (no detections), 12.5ms
Speed: 5.5ms preprocess, 12.5ms i


0: 480x800 1 fire - v1 2024-01-12 8-02pm, 13.5ms
Speed: 4.5ms preprocess, 13.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 3 fire - v1 2024-01-12 8-02pms, 12.1ms
Speed: 4.5ms preprocess, 12.1ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 2 fire - v1 2024-01-12 8-02pms, 13.1ms
Speed: 3.0ms preprocess, 13.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 3 fire - v1 2024-01-12 8-02pms, 11.7ms
Speed: 4.1ms preprocess, 11.7ms inference, 4.2ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 3 fire - v1 2024-01-12 8-02pms, 10.7ms
Speed: 3.6ms preprocess, 10.7ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 3 fire - v1 2024-01-12 8-02pms, 11.0ms
Speed: 4.9ms preprocess, 11.0ms inference, 5.1ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 3 fire - v1 2024-01-12 8-02pms, 11.9ms
Speed: 4.0ms preprocess, 11.9ms inference, 2.0ms postprocess p


0: 480x800 1 fire - v1 2024-01-12 8-02pm, 12.0ms
Speed: 3.1ms preprocess, 12.0ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fire - v1 2024-01-12 8-02pm, 12.5ms
Speed: 3.0ms preprocess, 12.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fire - v1 2024-01-12 8-02pm, 11.4ms
Speed: 3.0ms preprocess, 11.4ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fire - v1 2024-01-12 8-02pm, 13.0ms
Speed: 2.0ms preprocess, 13.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fire - v1 2024-01-12 8-02pm, 11.5ms
Speed: 3.0ms preprocess, 11.5ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fire - v1 2024-01-12 8-02pm, 11.2ms
Speed: 3.9ms preprocess, 11.2ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fire - v1 2024-01-12 8-02pm, 12.5ms
Speed: 2.9ms preprocess, 12.5ms inference, 1.5ms postprocess per ima


0: 480x800 1 fire - v1 2024-01-12 8-02pm, 12.6ms
Speed: 2.5ms preprocess, 12.6ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fire - v1 2024-01-12 8-02pm, 12.1ms
Speed: 3.0ms preprocess, 12.1ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fire - v1 2024-01-12 8-02pm, 13.2ms
Speed: 2.6ms preprocess, 13.2ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fire - v1 2024-01-12 8-02pm, 11.5ms
Speed: 3.0ms preprocess, 11.5ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fire - v1 2024-01-12 8-02pm, 11.5ms
Speed: 3.0ms preprocess, 11.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fire - v1 2024-01-12 8-02pm, 11.0ms
Speed: 4.0ms preprocess, 11.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fire - v1 2024-01-12 8-02pm, 11.7ms
Speed: 3.0ms preprocess, 11.7ms inference, 2.0ms postprocess per ima


0: 480x800 1 fire - v1 2024-01-12 8-02pm, 12.0ms
Speed: 2.5ms preprocess, 12.0ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fire - v1 2024-01-12 8-02pm, 10.5ms
Speed: 3.5ms preprocess, 10.5ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fire - v1 2024-01-12 8-02pm, 11.0ms
Speed: 6.0ms preprocess, 11.0ms inference, 3.6ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fire - v1 2024-01-12 8-02pm, 12.5ms
Speed: 4.0ms preprocess, 12.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fire - v1 2024-01-12 8-02pm, 11.5ms
Speed: 2.7ms preprocess, 11.5ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fire - v1 2024-01-12 8-02pm, 10.5ms
Speed: 3.5ms preprocess, 10.5ms inference, 3.1ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fire - v1 2024-01-12 8-02pm, 11.0ms
Speed: 3.3ms preprocess, 11.0ms inference, 2.0ms postprocess per ima


0: 480x800 1 fire - v1 2024-01-12 8-02pm, 12.3ms
Speed: 3.0ms preprocess, 12.3ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fire - v1 2024-01-12 8-02pm, 12.5ms
Speed: 4.0ms preprocess, 12.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fire - v1 2024-01-12 8-02pm, 11.5ms
Speed: 3.0ms preprocess, 11.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fire - v1 2024-01-12 8-02pm, 12.5ms
Speed: 3.0ms preprocess, 12.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fire - v1 2024-01-12 8-02pm, 11.5ms
Speed: 3.5ms preprocess, 11.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fire - v1 2024-01-12 8-02pm, 11.5ms
Speed: 3.0ms preprocess, 11.5ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fire - v1 2024-01-12 8-02pm, 11.0ms
Speed: 3.0ms preprocess, 11.0ms inference, 2.0ms postprocess per ima


0: 480x800 1 fire - v1 2024-01-12 8-02pm, 12.5ms
Speed: 4.1ms preprocess, 12.5ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fire - v1 2024-01-12 8-02pm, 11.1ms
Speed: 3.4ms preprocess, 11.1ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fire - v1 2024-01-12 8-02pm, 11.5ms
Speed: 3.5ms preprocess, 11.5ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fire - v1 2024-01-12 8-02pm, 10.3ms
Speed: 4.0ms preprocess, 10.3ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fire - v1 2024-01-12 8-02pm, 12.0ms
Speed: 3.0ms preprocess, 12.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fire - v1 2024-01-12 8-02pm, 12.5ms
Speed: 3.0ms preprocess, 12.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 fire - v1 2024-01-12 8-02pm, 11.7ms
Speed: 3.8ms preprocess, 11.7ms inference, 2.0ms postprocess per ima


0: 480x800 1 fire - v1 2024-01-12 8-02pm, 12.1ms
Speed: 3.0ms preprocess, 12.1ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 (no detections), 11.7ms
Speed: 3.0ms preprocess, 11.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 (no detections), 11.2ms
Speed: 3.8ms preprocess, 11.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 (no detections), 10.9ms
Speed: 6.0ms preprocess, 10.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 (no detections), 11.3ms
Speed: 2.5ms preprocess, 11.3ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 (no detections), 11.5ms
Speed: 4.0ms preprocess, 11.5ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 (no detections), 11.0ms
Speed: 4.0ms preprocess, 11.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 (no detections), 11.1ms
Speed: 4.5ms prepro

The video was successfully processed and saved as ../results\video1_detections.mp4.


## Realtime pred on web cam

In [12]:
import cv2
from ultralytics import YOLO

# Initialize the YOLO model
model_path = "../models/best.pt"
model = YOLO(model_path)

# Open a connection to the webcam
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Perform fire detection on the current frame
    results = model.predict([frame])  # Get predictions for the current frame

    # Assuming 'results[0].boxes' contains the detection results
    detections = results[0].boxes

    # Check if any detections were made
    if detections.data.shape[0] > 0:
        for i in range(detections.data.shape[0]):
            box = detections.data[i]
            x1, y1, x2, y2 = map(int, box[:4])  # Extract bounding box coordinates
            conf = box[4].item()  # Confidence score

            # Draw the bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 4)

            # Prepare the label as "Fire" and the confidence score
            label = f'Fire: {conf:.2f}'

            # Calculate text width & height to draw the transparent box as background of the text
            (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)

            # Display the label at the top of the bounding box
            label_position = (x1, y1 - 15 if y1 - 15 > 15 else y1 + 15)
            cv2.rectangle(frame, (x1, label_position[1] - h - 3), (x1 + w, label_position[1] + h - 3), (0, 255, 0), -1)
            cv2.putText(frame, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 0), 2)

    # Display the resulting frame with detections
    cv2.imshow('Real-Time Fire Detection', frame)

    # Break the loop with the 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
cap.release()
cv2.destroyAllWindows()



0: 608x800 (no detections), 17.1ms
Speed: 3.0ms preprocess, 17.1ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 15.0ms
Speed: 4.0ms preprocess, 15.0ms inference, 2.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 15.2ms
Speed: 4.0ms preprocess, 15.2ms inference, 2.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 14.9ms
Speed: 4.0ms preprocess, 14.9ms inference, 2.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 15.7ms
Speed: 3.3ms preprocess, 15.7ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 16.0ms
Speed: 3.3ms preprocess, 16.0ms inference, 1.2ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 15.1ms
Speed: 4.0ms preprocess, 15.1ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 15.9ms
Speed: 3.0ms preprocess, 15.9ms i


0: 608x800 (no detections), 17.4ms
Speed: 3.0ms preprocess, 17.4ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 17.0ms
Speed: 3.0ms preprocess, 17.0ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 15.6ms
Speed: 3.5ms preprocess, 15.6ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 15.5ms
Speed: 3.1ms preprocess, 15.5ms inference, 1.5ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 14.7ms
Speed: 3.4ms preprocess, 14.7ms inference, 1.2ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 16.2ms
Speed: 2.7ms preprocess, 16.2ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 15.0ms
Speed: 3.5ms preprocess, 15.0ms inference, 1.5ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 15.1ms
Speed: 3.1ms preprocess, 15.1ms i


0: 608x800 (no detections), 12.4ms
Speed: 3.5ms preprocess, 12.4ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.5ms
Speed: 4.5ms preprocess, 12.5ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.5ms
Speed: 3.6ms preprocess, 12.5ms inference, 0.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.5ms
Speed: 3.0ms preprocess, 12.5ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.8ms
Speed: 4.0ms preprocess, 12.8ms inference, 1.5ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.0ms
Speed: 3.6ms preprocess, 12.0ms inference, 2.1ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 13.5ms
Speed: 2.7ms preprocess, 13.5ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.5ms
Speed: 3.5ms preprocess, 12.5ms i


0: 608x800 (no detections), 12.5ms
Speed: 3.0ms preprocess, 12.5ms inference, 2.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.8ms
Speed: 3.0ms preprocess, 12.8ms inference, 0.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.2ms
Speed: 3.5ms preprocess, 12.2ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 13.8ms
Speed: 4.5ms preprocess, 13.8ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.5ms
Speed: 4.1ms preprocess, 12.5ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.6ms
Speed: 3.3ms preprocess, 12.6ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.5ms
Speed: 3.2ms preprocess, 12.5ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.2ms
Speed: 3.1ms preprocess, 12.2ms i


0: 608x800 (no detections), 13.1ms
Speed: 3.0ms preprocess, 13.1ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.7ms
Speed: 3.5ms preprocess, 12.7ms inference, 1.2ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.5ms
Speed: 3.5ms preprocess, 12.5ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 13.5ms
Speed: 4.0ms preprocess, 13.5ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.5ms
Speed: 3.5ms preprocess, 12.5ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.1ms
Speed: 3.2ms preprocess, 12.1ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.2ms
Speed: 3.5ms preprocess, 12.2ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 13.1ms
Speed: 2.9ms preprocess, 13.1ms i

...


0: 608x800 (no detections), 13.0ms
Speed: 3.5ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.8ms
Speed: 3.0ms preprocess, 12.8ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 13.0ms
Speed: 3.1ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.5ms
Speed: 3.1ms preprocess, 12.5ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 13.0ms
Speed: 3.5ms preprocess, 13.0ms inference, 1.1ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.5ms
Speed: 4.0ms preprocess, 12.5ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.5ms
Speed: 4.0ms preprocess, 12.5ms inference, 1.5ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.5ms
Speed: 3.2ms preprocess, 12.5ms i


0: 608x800 (no detections), 12.9ms
Speed: 3.1ms preprocess, 12.9ms inference, 1.1ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.5ms
Speed: 3.1ms preprocess, 12.5ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.5ms
Speed: 2.4ms preprocess, 12.5ms inference, 1.5ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 13.0ms
Speed: 3.3ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.9ms
Speed: 3.0ms preprocess, 12.9ms inference, 1.4ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.2ms
Speed: 3.8ms preprocess, 12.2ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 13.1ms
Speed: 3.5ms preprocess, 13.1ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.5ms
Speed: 4.0ms preprocess, 12.5ms i


0: 608x800 (no detections), 12.5ms
Speed: 4.0ms preprocess, 12.5ms inference, 1.5ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.9ms
Speed: 3.1ms preprocess, 12.9ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.2ms
Speed: 4.0ms preprocess, 12.2ms inference, 1.1ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.5ms
Speed: 3.7ms preprocess, 12.5ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.1ms
Speed: 3.5ms preprocess, 12.1ms inference, 1.1ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 13.1ms
Speed: 3.0ms preprocess, 13.1ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.4ms
Speed: 3.0ms preprocess, 12.4ms inference, 1.5ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.4ms
Speed: 4.0ms preprocess, 12.4ms i


0: 608x800 (no detections), 13.2ms
Speed: 3.4ms preprocess, 13.2ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.5ms
Speed: 4.4ms preprocess, 12.5ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.5ms
Speed: 3.0ms preprocess, 12.5ms inference, 2.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 13.0ms
Speed: 3.5ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 14.0ms
Speed: 3.1ms preprocess, 14.0ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 13.1ms
Speed: 3.0ms preprocess, 13.1ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.4ms
Speed: 3.5ms preprocess, 12.4ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.9ms
Speed: 3.5ms preprocess, 12.9ms i


0: 608x800 (no detections), 12.9ms
Speed: 3.1ms preprocess, 12.9ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.5ms
Speed: 3.1ms preprocess, 12.5ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.0ms
Speed: 3.0ms preprocess, 12.0ms inference, 2.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 12.4ms
Speed: 3.5ms preprocess, 12.4ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 13.1ms
Speed: 3.5ms preprocess, 13.1ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 13.0ms
Speed: 4.6ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 13.0ms
Speed: 3.5ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 13.5ms
Speed: 3.0ms preprocess, 13.5ms i

## validating the model on val dataset

In [19]:
# !yolo task=detect mode=val model=/content/datasets/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml

In [23]:
!yolo task=detect mode=val model=../models/best.pt data=../dataset/data2.yaml

Ultralytics YOLOv8.0.196  Python-3.10.0 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

val: Scanning C:\Users\Jarvis\Desktop\Web\Fire_Detection\dataset\valid\labels...:   0%|          | 0/938 [00:00<?, ?it/s]
val: Scanning C:\Users\Jarvis\Desktop\Web\Fire_Detection\dataset\valid\labels... 79 images, 0 backgrounds, 0 corrupt:   8%|8         | 79/938 [00:00<00:01, 773.80it/s]
val: Scanning C:\Users\Jarvis\Desktop\Web\Fire_Detection\dataset\valid\labels... 186 images, 0 backgrounds, 0 corrupt:  20%|#9        | 186/938 [00:00<00:00, 944.29it/s]
val: Scanning C:\Users\Jarvis\Desktop\Web\Fire_Detection\dataset\valid\labels... 286 images, 0 backgrounds, 0 corrupt:  30%|###       | 286/938 [00:00<00:00, 964.74it/s]
val: Scanning C:\Users\Jarvis\Desktop\Web\Fire_Detection\dataset\valid\labels... 383 images, 0 backgrounds, 0 corrupt:  41%|####      | 383/938 [00:00<00:00, 960.68it/s]
val: Sc